Load workspace - we need this to be able to download the data

In [2]:
# load data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Load Packages - muss ich noch aufräumen

In [3]:
#from minisom import MiniSom

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import warnings
import sys
import gzip
import gensim 
import logging
from tqdm import tqdm
!pip install minisom
!pip install nltk
from minisom import MiniSom
import string
from gensim.models.doc2vec import Doc2Vec
import nltk
from nltk import RegexpTokenizer
from nltk.tokenize.casual import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from random import randint
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
 
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

%matplotlib inline

  Stored in directory: /root/.cache/pip/wheels/08/46/49/a920ae8083e5da81a42e90a799fb73a0bc52006d927197e50b
Successfully built minisom
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
pd.get_option('max_colwidth')
pd.set_option('max_colwidth', 2000)

In [0]:
data =pd.read_csv('/content/gdrive/My Drive/out.csv', sep=',').iloc[:, 1:]
data

Cleaning the data: delete special characters, stopwords

In [7]:
data['tweet'] = data['tweet'].apply(lambda x: re.sub('([\.\",\(\)!\?;:])[!@#$:+).;,?&]1234567890/', '', x.lower()))
data['tweet'] = data['tweet'].apply(lambda x: re.sub('  ', ' ', x))
data['tweet'][1]

'rt @bobcdu: in #paris hat @akk auch nochmals jegliche zusammenarbeit mit der rechtspopulistischen afd kategorisch ausgeschlossen. rechtspop…'

Some summary statistics

In [8]:
data['word_count'] = data['tweet'].str.count(' ') + 1
# words per tweet
data.groupby('party')['word_count'].mean()

party
AfD      18.000000
CDU      16.567568
FDP      14.500000
Grüne    18.800000
Linke    17.700000
SPD      18.675000
Name: word_count, dtype: float64

In [0]:
# words per tweet and politician
data.groupby('real')['word_count'].mean()

work in progress: müssen irgendwie noch die stopwords in englisch und deutsch entfernen aber geht gerade nicht so gut

In [11]:
#Posts per Politician
data['real'].value_counts()

Sarah Wagenknecht              20
Jörg Meuthen                   20
Cem Özdemir                    20
Christian Lindner              20
Annegreth Kramp-Karrenbauer    20
Katharina Barley               20
Kathrin Goering-Eckardt        20
Alice Weidel                   20
Andrea Nahles                  20
Angela Merkel                  17
Name: real, dtype: int64

Remove stopwords, links, special characters

In [0]:
stopwords.words('german')[:50]

In [0]:
# remove stopwords
stop = set(stopwords.words('german', 'english')) 
data['newtweet'] = data['tweet'].str.split()
data['newtweet'] = data['newtweet'].apply(lambda x : [item for item in x if item not in stop])
data["newtweet"]= data["newtweet"].str.join(" ") 

In [0]:
data['newtweet']

In [0]:
# remove urls
data['newtweet'] = data['newtweet'].str.replace('http\S+|www.\S+', '', case=False)


Tag Data: helps to identify semantic structures in sentences, necessary for our doc2vec training
source: https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

In [0]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data['newtweet'])]
tagged_data

In [16]:
tagged_data = [w for w in tagged_data if not w in stop_words] 

TypeError: ignored

Train our Doc2Vec model -> i think we need to readjust the parameters
The model uses a neural network to depict similarity between sentences (in our case between tweets) as a numerical vector, which is exactly what we need as input for our SOM!



In [0]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Load our trained model

In [0]:
model= Doc2Vec.load("d2v.model")

2019-06-21 12:45:57,673 : INFO : loading Doc2Vec object from d2v.model
2019-06-21 12:45:57,691 : INFO : loading vocabulary recursively from d2v.model.vocabulary.* with mmap=None
2019-06-21 12:45:57,693 : INFO : loading trainables recursively from d2v.model.trainables.* with mmap=None
2019-06-21 12:45:57,694 : INFO : loading wv recursively from d2v.model.wv.* with mmap=None
2019-06-21 12:45:57,699 : INFO : loading docvecs recursively from d2v.model.docvecs.* with mmap=None
2019-06-21 12:45:57,701 : INFO : loaded d2v.model


In [0]:
# fetch value of vector of first tweet
print(model.docvecs['1'])


In [0]:
# number of words in our vocabulary
len(model.wv.vocab)

In [0]:
# number of trained document tags
len(model.docvecs)

In [0]:
# length of word vector
len(model.docvecs[1])

In [0]:
wordvector = pd.DataFrame(model.docvecs[1], ["x1", "x2", "x3", "x4", "x5",
                  "x6", "x7", "x8", "x9", "x10",
                  "x11", "x12", "x13", "x14", "x15",
                  "x16", "x17", "x18", "x19", "x20"])
wordvector


In [0]:
for col in range(1, len(data)): 
 wordvector[col] = pd.DataFrame(model.docvecs[col], ["x1", "x2", "x3", "x4", "x5",
                  "x6", "x7", "x8", "x9", "x10",
                  "x11", "x12", "x13", "x14", "x15",
                  "x16", "x17", "x18", "x19", "x20"])
wordvector

In [0]:
joint_data = pd.concat([data, wordvector.T ], axis=1, sort=False)
joint_data

In [0]:
joint_data.to_csv('data_wordvectors.csv', sep=',', encoding='utf-8', index=True)

**TRASH CODE**

In [0]:
stopword_set = set(stopwords.words('german'))
tokenizer = TweetTokenizer
MessageDoc = namedtuple('MessageDoc', 'words tags')

alldocs = []  # Will hold all doacs in original order
for line_no, line in data.iterrows():
    message = line.message.lower()
    message = tokenizer().tokenize(message) 
    #import pdb; pdb.set_trace()
    words = [word.strip('#') for word in message if (word not in stopword_set 
                                          and word not in string.punctuation
                                          and not word.startswith('http')
                                          and not re.search(r'\d', word))]
    tags = [str(line_no), line['from_name']] #, line['Partei_ABK']] # line_no needs to be converted as string to be included in tags 
    alldocs.append(MessageDoc(words, tags))


In [0]:

from collections import OrderedDict

model = gensim.models.Doc2Vec(dm=0, dbow_words=0, size=100, negative=5, hs=0, min_count=5, workers=cores),
  

In [0]:
for name, train_model in models_by_name.items():
    train_model.train(alldocs, total_examples=len(alldocs), epochs=epochs, start_alpha=0.025, end_alpha=0.001)


NameError: ignored

In [0]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
documents = nlp_clean(tweets['text'])

In [0]:
# Choose a random document/candidate
i = randint(0, 5-1)
i
tweets.iloc[i,:]

Name                                     Katharina Barley
text    ['Herzlichen Glückwunsch liebe Christine #Lamb...
Name: 2, dtype: object

In [0]:
# clean tweets from special characters
tweets['text'] = tweets['text'].apply(lambda x: re.sub('([\.\",\(\)!\?;:])[!@#$:+).;,?&]1234567890/', '', x.lower()))
tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text'][1]


In [0]:
# produce corpus with words in format for gensim Word2vec
tmp_corpus = tweets['text'].map(lambda x: x.split('.'))
tmp_corpus 
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

In [0]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

In [0]:
#model = gensim.models.Doc2Vec(
        #corpus,
        #size=150,
        #window=10,
        #min_count=2,
        #workers=5,
       # iter=10)
 model = Doc2Vec(corpus, vector_size=5, window=2, min_count=1, workers=4)
 

In [0]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]

In [0]:
documents

In [0]:
wl = "heute"
model.wv.most_similar (positive = wl)

In [0]:
len(model.wv.vocab)
model.wv.vocab

In [0]:
som = MiniSom(6, 6, 4, sigma=0.3, learning_rate=0.5)
som.pca_weights_init(tweets)
print("Training...")
som.train_random(tweets, 100, verbose=True)
print("\n...ready!")

In [0]:
model.wv['heute']

In [0]:
map_dim = 16
som = MiniSom(map_dim, map_dim, 50, sigma=1.0, random_seed=1)
som.train_batch(W,5)

In [0]:
from sklearn import datasets
from sklearn.preprocessing import scale

# load the digits dataset from scikit-learn
digits = datasets.load_digits(n_class=10)
digits


In [0]:
data = scale(data)
num = digits.target  # num[i] is the digit represented by data[i]
data